## SCDV

In [29]:
import time
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer,HashingVectorizer

In [208]:
from gensim.models import Word2Vec
sentences = [['this', 'is', 'the', 'good', 'machine', 'learning', 'book'],
            ['this', 'is',  'another', 'book'],
            ['one', 'more', 'book'],
            ['this', 'is', 'the', 'new', 'post'],
            ['this', 'is', 'about', 'machine', 'learning', 'post'],  
            ['and', 'this', 'is', 'the', 'last', 'post']]
model = Word2Vec(sentences, min_count=1, size = 5, sg = 1) # based on skip-gram model

D:\anaconda\lib\site-packages\gensim\models\base_any2vec.py:743: UserWarning: C extension not loaded, training will be slow. Install a C compiler and reinstall gensim for fast training.
  "C extension not loaded, training will be slow. "


* **Build Clusters -- remember to change the directories**

In [209]:
word_vectors = model.wv.vectors

In [201]:
'''
def drange(start, stop, step):
    r = start
    while r < stop:
        yield r
        r += step
'''

'\ndef drange(start, stop, step):\n    r = start\n    while r < stop:\n        yield r\n        r += step\n'

In [18]:
def cluster_GMM(num_clusters, word_vectors):
    start = time.time()
    # Initalize a GMM object and use it for clustering.
    clf =  GaussianMixture(n_components=num_clusters,
                    covariance_type="tied", init_params='kmeans', max_iter=50)
    # Get cluster assignments.
    clf.fit(word_vectors)
    idx = clf.predict(word_vectors)
    print ("Clustering Done...", time.time()-start, "seconds")
    # Get probabilities of cluster assignments.
    idx_proba = clf.predict_proba(word_vectors)
    # Dump cluster assignments and probability of cluster assignments. 
    pickle.dump(idx, open('C:/Users/zhuchen/Desktop/motohashi seminar/Word2Vec Learning/gmm_latestclusmodel_len2alldata.pkl',"wb"))
    print ("Cluster Assignments Saved...")

    pickle.dump(idx_proba,open( 'C:/Users/zhuchen/Desktop/motohashi seminar/Word2Vec Learning/gmm_prob_latestclusmodel_len2alldata.pkl',"wb"))
    print ("Probabilities of Cluster Assignments Saved...")
    return (idx, idx_proba)

In [ ]:
def read_GMM(idx_name, idx_proba_name):
    # Loads cluster assignments and probability of cluster assignments. 
    idx = pickle.load(open('C:/Users/zhuchen/Desktop/motohashi seminar/Word2Vec Learning/gmm_latestclusmodel_len2alldata.pkl',"rb"))
    idx_proba = pickle.load(open( 'C:/Users/zhuchen/Desktop/motohashi seminar/Word2Vec Learning/gmm_prob_latestclusmodel_len2alldata.pkl',"rb"))
    print ("Cluster Model Loaded...")
    return (idx, idx_proba)

In [202]:
num_clusters = 2
(idx, idx_proba) = cluster_GMM(num_clusters, word_vectors)

Clustering Done... 0.7695479393005371 seconds
Cluster Assignments Saved...
Probabilities of Cluster Assignments Saved...


In [27]:
word_centroid_map = dict(zip(model.wv.index2word, idx))
word_centroid_prob_map = dict(zip(model.wv.index2word, idx_proba))

In [36]:
sentences_idf_ready = [' '.join(sentence) for sentence in sentences]
sentences_idf_ready

['this is the good machine learning book',
 'this is another book',
 'one more book',
 'this is the new post',
 'this is about machine learning post',
 'and this is the last post']

In [39]:
tfv = TfidfVectorizer()
tfidfmatrix = tfv.fit_transform(sentences_idf_ready)
featurenames = tfv.get_feature_names()
idf = tfv._tfidf.idf_

In [44]:
word_idf_dict = dict(zip(featurenames, idf))

In [100]:
def get_probability_word_vectors(model, featurenames, word_centroid_map, word_centroid_prob_map, word_idf_dict, num_features, num_clusters):
    # This function computes probability word-cluster vectors.
    # word_centroid_map: (word, cluster_id)
    # word_centroid_prob_map: (word, cluster_prob)
    prob_wordvecs = {}
    
    for word in word_centroid_map:
        prob_wordvecs[word] = np.zeros( num_clusters * num_features, dtype="float32" )
        for index in range(0, num_clusters):
            try:
                prob_wordvecs[word][index*num_features:(index+1)*num_features] = model.wv[word] * word_centroid_prob_map[word][index] * word_idf_dict[word]
            except:
                continue


    return prob_wordvecs

In [103]:
num_clusters = 2
num_features = 5
prob_wordvecs = get_probability_word_vectors(model, featurenames, word_centroid_map, word_centroid_prob_map, word_idf_dict, num_features, num_clusters)

In [124]:
def create_cluster_vector_and_gwbowv(prob_wordvecs, wordlist, word_centroid_map, word_centroid_prob_map, dimension, word_idf_dict, featurenames, num_centroids, train=False):
    # This function computes SDV feature vectors.
    bag_of_centroids = np.zeros( num_centroids * dimension, dtype="float32" )
    global min_no
    global max_no

    # wordlist : a document consists of a list of tokens
    for word in wordlist:
        try:
            temp = word_centroid_map[word]
        except:
            continue

        bag_of_centroids += prob_wordvecs[word]
        
    norm = np.sqrt(np.einsum('...i,...i', bag_of_centroids, bag_of_centroids))
    if(norm!=0):
        bag_of_centroids /= norm

    # To make feature vector sparse, make note of minimum and maximum values.
    if train:
        min_no += min(bag_of_centroids)
        max_no += max(bag_of_centroids)

    return bag_of_centroids

In [196]:
# build SCDV features
num_clusters = 2
num_features = 5
dimension = num_features
num_centroids = num_clusters
# sparsity setting
min_no = 0
max_no = 0
# run
doc_scdv = [create_cluster_vector_and_gwbowv(prob_wordvecs, doc, word_centroid_map, word_centroid_prob_map, dimension, word_idf_dict, featurenames, num_centroids, train=True) \
 for doc in sentences]
doc_scdv = np.vstack(doc_scdv)

In [193]:
# Set the threshold percentage for making it sparse
percentage = 0.04
print(len(doc_scdv))
min_no = min_no*1.0/len(doc_scdv)
max_no = max_no*1.0/len(doc_scdv)
print('Average min: ', min_no)
print('Averge max: ', max_no)
threshold = (abs(min_no) + abs(max_no))/2
threshold = threshold * percentage

6
Average min:  -0.558233305811882
Averge max:  0.4659252464771271


In [198]:
temp = abs(doc_scdv) < threshold
doc_scdv[temp] = 0
doc_scdv

array([[ 0.3607643 ,  0.2232363 ,  0.34561348,  0.5220664 , -0.5360979 ,
         0.        ,  0.24476816,  0.        ,  0.        ,  0.        ],
       [ 0.5010719 , -0.6639076 , -0.26762426, -0.43843392, -0.21049167,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.36032736, -0.24551527, -0.26580805,  0.57196534,  0.641941  ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.29085016,  0.        , -0.21184976,  0.        , -0.7502883 ,
         0.        ,  0.3337964 ,  0.        ,  0.30128208, -0.24067657],
       [ 0.        ,  0.        ,  0.26648086,  0.35002476, -0.38150784,
         0.28476605,  0.3804079 , -0.21408658,  0.46766   , -0.40083453],
       [ 0.        , -0.29343304,  0.        , -0.7324635 , -0.23919412,
         0.        ,  0.32901576,  0.        ,  0.2969671 , -0.23722959]],
      dtype=float32)

In [199]:
# save it
#np.save("../japanese-dataset/livedoor-news-corpus/model/"+gwbowv_name, gwbowv)